In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# Fix pybel import path
try:
    import sys
    sys.modules['pybel'] = __import__('openbabel', globals(), locals(), ['pybel']).pybel
except Exception:
    pass

In [ ]:
import ipywidgets as ipw
from jinja2 import Environment
from importlib_resources import files

from aiida.orm import StructureData
from aiida.orm import load_node
from aiidalab_widgets_base import OptimadeQueryWidget
from aiidalab_widgets_base import StructureBrowserWidget, StructureManagerWidget, StructureUploadWidget
from aiidalab_widgets_base import WizardAppWidget
from aiidalab_widgets_base.bug_report import install_create_github_issue_exception_handler

from aiidalab_qe import static
from aiidalab_qe.process import WorkChainSelector
from aiidalab_qe.steps import SubmitQeAppWorkChainStep
from aiidalab_qe.steps import ViewQeAppWorkChainStatusAndResultsStep
from aiidalab_qe.structures import StructureSelectionStep


OptimadeQueryWidget.title = "OPTIMADE"  # monkeypatch


# Create the application steps
structure_manager_widget = StructureManagerWidget(
    importers=[
        StructureUploadWidget(title="Upload file"),
        OptimadeQueryWidget(embedded=False),
        StructureBrowserWidget(title="AiiDA database"),
    ],
    node_class='StructureData',
    storable=False,
)
structure_selection_step = StructureSelectionStep(manager=structure_manager_widget)
structure_selection_step.auto_advance = True
submit_qe_app_work_chain_step = SubmitQeAppWorkChainStep()
submit_qe_app_work_chain_step.auto_advance = True
view_qe_app_work_chain_status_and_results_step = ViewQeAppWorkChainStatusAndResultsStep()

# Link the application steps
ipw.dlink((structure_selection_step, 'confirmed_structure'), (submit_qe_app_work_chain_step, 'input_structure'))
ipw.dlink((submit_qe_app_work_chain_step, 'process'), (view_qe_app_work_chain_status_and_results_step, 'process'))

# Add the application steps to the application
app = WizardAppWidget(
    steps=[
        ('Select structure', structure_selection_step),
        ('Submit work chain', submit_qe_app_work_chain_step),
        ('Status & Results', view_qe_app_work_chain_status_and_results_step),
    ])

# Reset all subsequent steps in case that a new structure is selected
def _observe_structure_selection(change):
    with structure_selection_step.hold_sync():
        if structure_selection_step.confirmed_structure is not None and \
                structure_selection_step.confirmed_structure != change['new']:
            app.reset()
structure_selection_step.observe(_observe_structure_selection, 'structure')

# Add process selection header
work_chain_selector = WorkChainSelector(layout=ipw.Layout(width='auto'))

def _observe_process_selection(change):
    if change['old'] == change['new']:
        return
    pk = change['new']
    if pk is None:
        app.reset()
        app.selected_index = 0
    else:
        process = load_node(pk)
        with structure_manager_widget.hold_sync():
            with structure_selection_step.hold_sync():
                structure_manager_widget.structure = process.inputs.structure
                structure_selection_step.structure = process.inputs.structure
                structure_selection_step.confirmed_structure = process.inputs.structure
                submit_qe_app_work_chain_step.process = process
        
work_chain_selector.observe(_observe_process_selection, 'value')    
ipw.dlink((submit_qe_app_work_chain_step, 'process'), (work_chain_selector, 'value'),
          transform=lambda node: None if node is None else node.pk)

env = Environment()

template = files(static).joinpath("welcome.jinja").read_text()
style = files(static).joinpath("style.css").read_text()
welcome_message = ipw.HTML(env.from_string(template).render(style=style))

app_with_work_chain_selector = ipw.VBox(children=[work_chain_selector, app])

output = ipw.Output()
install_create_github_issue_exception_handler(
    output,
    url='https://github.com/aiidalab/aiidalab-qe/issues/new',
    labels=('bug', 'automated-report'))

with output:
    display(welcome_message, app_with_work_chain_selector)
    
display(output)